In [1]:
import pandas as pd
import sklearn

In [172]:
# load data
train_identity = pd.read_csv(r'.\ieee-fraud-detection\train_identity.csv')
train_transaction = pd.read_csv(r'.\ieee-fraud-detection\train_transaction.csv')
test_identity = pd.read_csv(r'.\ieee-fraud-detection\test_identity.csv')
test_transaction = pd.read_csv(r'.\ieee-fraud-detection\test_transaction.csv')

In [181]:
# left join on TransactionID, store in train
train = pd.merge(train_transaction, train_identity, on = 'TransactionID', how = 'left')
test = pd.merge(test_transaction, test_identity, on = 'TransactionID', how = 'left')
train_label = train['isFraud']
train = train.drop(columns = ['isFraud'])

In [183]:
all_data = pd.concat([train, test], axis = 0)

In [184]:
# make a dictionary to store all dtypes of all features
dtype_dict = {}
for i in all_data.columns.tolist():
    dtype_dict[i]= train[i].dtype
# print unique data type
set(dtype_dict.values())

{dtype('int64'), dtype('float64'), dtype('O')}

In [185]:
# make a dictionary to store all feature type
# we know int and float are continuous O is categorical
# cont for continuous
# cat for catagorical
feature_type = {}
for i in dtype_dict.keys():
    if dtype_dict[i] == 'int64':
        feature_type[i] = 'cont'
    if dtype_dict[i] == 'float64':
        feature_type[i] = 'cont'
    if dtype_dict[i] == 'O':
        feature_type[i] = 'cat'
    

In [186]:
# find percentage of non null value of each feature
null = {}
for i in all_data.columns.tolist():
    null[i] = all_data[i].isnull().value_counts()[False]/all_data.shape[0]


In [191]:
# check the number of unique values of each categorical features
for i in all_data.columns.tolist():
    if feature_type[i] == 'cat':
        print('{} has {} unique values'.format(i,len(set(all_data[i].unique()))))

ProductCD has 5 unique values
card4 has 5 unique values
card6 has 5 unique values
P_emaildomain has 61 unique values
R_emaildomain has 61 unique values
M1 has 3 unique values
M2 has 3 unique values
M3 has 3 unique values
M4 has 4 unique values
M5 has 3 unique values
M6 has 3 unique values
M7 has 3 unique values
M8 has 3 unique values
M9 has 3 unique values
id_12 has 3 unique values
id_15 has 4 unique values
id_16 has 3 unique values
id_23 has 4 unique values
id_27 has 3 unique values
id_28 has 3 unique values
id_29 has 3 unique values
id_30 has 88 unique values
id_31 has 173 unique values
id_33 has 462 unique values
id_34 has 5 unique values
id_35 has 3 unique values
id_36 has 3 unique values
id_37 has 3 unique values
id_38 has 3 unique values
DeviceType has 3 unique values
DeviceInfo has 2800 unique values
